<a href="https://colab.research.google.com/github/kaz2ngt/colaboratory/blob/master/12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 事前準備: ドライブのマウントとデータの取得

In [0]:
# Google ドライブをマウントするには、このセルを実行してください。
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# 下記でdriveからコピーして解凍
!cp drive/My\ Drive/Colab\ Notebooks/12data.zip ./
!unzip 12data.zip

# リストPART2

In [0]:
import os
import glob
import math
import random

import numpy as np
import matplotlib.pyplot as plt

from tensorflow.python import keras
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model, Sequential
from tensorflow.python.keras.layers import Conv2D, Dense, Input, MaxPooling2D, UpSampling2D, Lambda
from tensorflow.python.keras.preprocessing.image import load_img, img_to_array, array_to_img, ImageDataGenerator

In [0]:
# 足りないimport追加
from tensorflow.python.keras.layers import Flatten, Reshape
from tensorflow.python.keras.optimizers import Adam

In [0]:
# 足りないmethod追加
def save_imgs(path, imgs, rows, cols):
    """画像をタイル状にならべて保存する

    Arguments:
        path (str): 保存先のファイルパス
        imgs (np.array): 保存する画像のリスト
        rows (int): タイルの縦のサイズ
        cols (int): タイルの横のサイズ

    Returns:
        None
    """
    base_width = imgs.shape[1]
    base_height = imgs.shape[2]
    channels = imgs.shape[3]
    output_shape = (
        base_height * rows,
        base_width * cols,
        channels
    )
    buffer = np.zeros(output_shape)
    for row in range(rows):
        for col in range(cols):
            img = imgs[row * cols + col]
            buffer[
            row * base_height:(row + 1) * base_height,
            col * base_width:(col + 1) * base_width
            ] = img
    array_to_img(buffer).save(path)

# 12_1: 画像データの読み込み

In [0]:
DATA_DIR = 'data/chap12/'
BATCH_SIZE = 16
IMG_SHAPE = (64, 64, 3)

data_gen = ImageDataGenerator(rescale=1/255.)
train_data_generator = data_gen.flow_from_directory(
    directory=DATA_DIR,
    classes=['faces'],
    class_mode=None,
    batch_size=BATCH_SIZE,
    target_size=IMG_SHAPE[:2]
)

Found 19370 images belonging to 1 classes.


# 12_2: Encoderの定義

In [0]:
def build_encoder(input_shape, z_size, n_filters, n_layers):
  """Encoderを構築する

  Arguments:
    input_shape (int): 画像のshape
    z_size (int): 特徴空間の次元数
    n_filters (int): フィルタ数

  Returns:
   model (Model): Encoderモデル
  """
  model = Sequential()
  model.add(
      Conv2D(
          n_filters,
          3,
          activation='elu',
          input_shape=input_shape,
          padding='same'
      )
  )
  model.add(Conv2D(n_filters, 3, padding='same'))
  for i in range(2, n_layers + 1):
    model.add(
        Conv2D(
            i*n_filters,
            3,
            activation='elu',
            padding='same'
        )
    )
    model.add(
        Conv2D(
            i*n_filters,
            3,
            activation='elu',
            strides=2,
            padding='same'
        )
    )
  model.add(Conv2D(n_layers*n_filters, 3, padding='same'))
  model.add(Flatten())
  model.add(Dense(z_size))

  return model

# 12_3: Generator/Decoderの定義

In [0]:
def build_decoder(output_shape, z_size, n_filters, n_layers):
  """Decoderを構築する

  Arguments:
    output_shape (np.array): 画像のshape
    z_site (int): 特徴空間の次元数
    n_filters (int): フィルタ数
    n_layers (int): レイヤー数

  Returns:
    model (Model):  Decoderモデル
  """
  # UpSampling2Dで何倍に拡大されるか
  scale = 2**(n_layers - 1)
  # 最初の畳み込み層の入力サイズをscaleから逆算
  fc_shape = (
      output_shape[0]//scale,
      output_shape[1]//scale,
      n_filters
 )
  # 全結合層で必要なサイズを逆算
  fc_size = fc_shape[0]*fc_shape[1]*fc_shape[2]

  model = Sequential()
  # 全結合層
  model.add(Dense(fc_size, input_shape=(z_size,)))
  model.add(Reshape(fc_shape))

  # 畳み込み層の繰り返し
  for i in range(n_layers - 1):
    model.add(
        Conv2D(
            n_filters,
            3,
            activation='elu',
            padding='same'
        )
    )
    model.add(
        Conv2D(
            n_filters,
            3,
            activation='elu',
            padding='same'
        )
    )
    model.add(UpSampling2D())

  # 最後の層はUpSampling2Dが不要
  model.add(
      Conv2D(
          n_filters,
          3,
          activation='elu',
          padding='same',
      )
  )
  model.add(
      Conv2D(
          n_filters,
          3,
          activation='elu',
          padding='same'
      )
  )
  # 出力層で3チャンネルに
  model.add(Conv2D(3, 3, padding='same'))

  return model

# 12_4: Generatorの定義

In [0]:
def build_generator(img_shape, z_size, n_filters, n_layers):
  decoder = build_decoder(
      img_shape, z_size, n_filters, n_layers
  )
  return decoder

# 12_5: Discriminatorの定義

In [0]:
def build_discriminator(img_shape, z_size, n_filters, n_layers):
  encoder = build_encoder(
      img_shape, z_size, n_filters, n_layers
  )
  decoder = build_decoder(
      img_shape, z_size, n_filters, n_layers
  )
  return Sequential((encoder, decoder))

# 12_6: Discriminatorの学習用のネットワーク

In [0]:
def build_discriminator_trainer(discriminator):
  img_shape = discriminator.input_shape[1:]
  real_inputs = Input(img_shape)
  fake_inputs = Input(img_shape)
  real_outputs = discriminator(real_inputs)
  fake_outputs = discriminator(fake_inputs)

  return Model(
      inputs=[real_inputs, fake_inputs],
      outputs=[real_outputs, fake_outputs]
  )

# 12_7: ネットワークの構築

In [0]:
n_filters = 64 # フィルタ数
n_layers = 4# レイヤー数
z_size = 32 # 特徴空間の次元数

generator = build_generator(
    IMG_SHAPE, z_size, n_filters, n_layers
)
discriminator = build_discriminator(
    IMG_SHAPE, z_size, n_filters, n_layers
)
discriminator_trainer  =  build_discriminator_trainer(
    discriminator
)

generator.summary()
# discriminator.layers[1]が Decoder を表す
discriminator.layers[1].summary()

W0828 00:03:59.768161 140560677394304 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 4096)              135168    
_________________________________________________________________
reshape (Reshape)            (None, 8, 8, 64)          0         
_________________________________________________________________
conv2d (Conv2D)              (None, 8, 8, 64)          36928     
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 8, 8, 64)          36928     
_________________________________________________________________
up_sampling2d (UpSampling2D) (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        36928     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 64)        3

# 12_8: 損失関数の定義

In [0]:
from tensorflow.python.keras.losses import mean_absolute_error

def build_generator_loss(discriminator):
  # discriminator を使って損失関数を定義
  def loss(y_true, y_pred):
    # y_true はダミー
    reconst = discriminator(y_pred)
    return mean_absolute_error(
        reconst,
        y_pred
    )
  return loss

# 12_9: generatorのコンパイル

In [0]:
# 初期の学習率(Generator)
g_lr = 0.0001

generator_loss = build_generator_loss(discriminator)
generator.compile(
    loss=generator_loss,
    optimizer=Adam(g_lr)
)

# 12_10: discriminatorのコンパイル

In [0]:
# 初期の学習率(Discriminator)
d_lr = 0.0001

# k_varは数値(普通の変数)
k_var = 0.0
# k はKeras(TensorFlow)のVariable
k = K.variable(k_var)
discriminator_trainer.compile(
    loss=[
        mean_absolute_error,
        mean_absolute_error
    ],
    loss_weights=[1., -k],
    optimizer=Adam(d_lr)
)

# 12_11: 収束判定用の関数定義

In [0]:
def measure(real_loss,  fake_loss,  gamma):
  return real_loss + np.abs(gamma*real_loss - fake_loss)

# 12_12: 学習のコード

In [0]:
# k の更新に利用するパラメータ
GAMMA = 0.5
LR_K = 0.001


# 繰り返し数。100000~1000000程度を指定
TOTAL_STEPS = 100000

# モデルや確認用の生成画像を保存するディレクトリ
MODEL_SAVE_DIR = 'began_s/models'
IMG_SAVE_DIR = 'began_s/imgs'
# 確認用に 5x5 個の画像を生成する
IMG_SAMPLE_SHAPE = (5, 5)
N_IMG_SAMPLES = np.prod(IMG_SAMPLE_SHAPE)

# 保存先がなければ作成
os.makedirs(MODEL_SAVE_DIR, exist_ok=True)
os.makedirs(IMG_SAVE_DIR, exist_ok=True)

# サンプル画像用のランダムシード
sample_seeds = np.random.uniform(
    -1, 1, (N_IMG_SAMPLES, z_size)
)

history = []
logs = []

for step, batch in enumerate(train_data_generator):
  # サンプル数が BATCH_SIZE に満たない場合はスキップ
  # 全体の画像枚数が BATCH_SIZE の倍数でない場合に発生
  if len(batch) < BATCH_SIZE:
    continue

  # 学習終了
  if step > TOTAL_STEPS:
    break

  # ランダムな値を生成
  z_g =  np.random.uniform(
      -1, 1, (BATCH_SIZE, z_size)
  )
  z_d =  np.random.uniform(
      -1, 1, (BATCH_SIZE, z_size)
  )

  # 生成画像 (discriminatorの学習に利用)
  g_pred = generator.predict(z_d)

  # generator を1ステップ分学習させる
  generator.train_on_batch(z_g, batch)
  # discriminator を1ステップ分学習させる
  _, real_loss, fake_loss = discriminator_trainer.train_on_batch(
      [batch, g_pred],
      [batch, g_pred]
  )
  # k を更新
  k_var += LR_K*(GAMMA*real_loss - fake_loss)
  K.set_value(k, k_var)

  # g_measure を計算するためにlossを保存
  history.append({
      'real_loss': real_loss,
      'fake_loss': fake_loss
  })

  # 1000回に1度ログを表示
  if step%1000 == 0:
    # 過去1000回分の measure を平均
    measurement = np.mean([
        measure(
            loss['real_loss'],
            loss['fake_loss'],
            GAMMA
        )
        for loss in history[-1000:]
    ])

    logs.append({
        'k': K.get_value(k),
        'measure': measurement,
        'real_loss': real_loss,
        'fake_loss': fake_loss
    })
    print(logs[-1])

    # 画像を保存
    img_path = '{}/generated_{}.png'.format(
        IMG_SAVE_DIR,
        step
    )
    save_imgs(
        img_path,
        generator.predict(sample_seeds),
        rows=IMG_SAMPLE_SHAPE[0],
        cols=IMG_SAMPLE_SHAPE[1]
    )
    # 最新のモデルを保存
    generator.save('{}/generator_{}.hd5'.format(MODEL_SAVE_DIR, step))
    discriminator.save('{}/discriminator_{}.hd5'.format(MODEL_SAVE_DIR, step))

{'k': 0.00012911911, 'measure': 0.4794679433107376, 'real_loss': 0.35034883, 'fake_loss': 0.046055302}
{'k': 0.035507627, 'measure': 0.145726781161502, 'real_loss': 0.08017315, 'fake_loss': 0.020892639}
{'k': 0.049880195, 'measure': 0.1126139170108363, 'real_loss': 0.09410982, 'fake_loss': 0.04782919}
{'k': 0.040487435, 'measure': 0.11290999461524188, 'real_loss': 0.0852791, 'fake_loss': 0.057748795}
{'k': 0.038235635, 'measure': 0.10297443783655763, 'real_loss': 0.100247905, 'fake_loss': 0.051553573}
{'k': 0.0448657, 'measure': 0.09864948162436485, 'real_loss': 0.08979483, 'fake_loss': 0.03644176}
{'k': 0.04934461, 'measure': 0.09310711471736431, 'real_loss': 0.09122168, 'fake_loss': 0.040378205}
{'k': 0.047206722, 'measure': 0.09947422783821822, 'real_loss': 0.09506133, 'fake_loss': 0.042102866}
{'k': 0.025358822, 'measure': 0.12962548524700104, 'real_loss': 0.0931938, 'fake_loss': 0.033776127}
{'k': 0.04150757, 'measure': 0.10226894765719771, 'real_loss': 0.07868658, 'fake_loss': 0.